In [1]:
import numpy as np
from openfermion.transforms import get_sparse_operator
from openfermion.utils import pauli_exp_to_qasm
from openfermion.ops import QubitOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from pyscf import ci
from pyscf import mp
from openfermionpyscf import run_pyscf
from openfermion.hamiltonians import MolecularData
import os

from numpy import array, concatenate, zeros
from numpy.random import randn
from scipy.optimize import minimize

from openfermion.config import *

from openfermion.hamiltonians import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.utils import uccsd_singlet_paramsize

In [2]:
geometry = [["H", [0, 0, 0]],
            ["H", [0, 0, 0.7414]]]
basis = "sto-3g"
multiplicity = 1
charge = 0
molecule = MolecularData(geometry, basis, multiplicity, charge)
h2_molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)

In [3]:
orbitals = h2_molecule.n_orbitals
qubit= h2_molecule.n_qubits
elektron=h2_molecule.n_electrons

In [4]:
from openfermion.utils import uccsd_singlet_generator, uccsd_generator
initial_amplitudes = np.linspace(0,0.05677,2)
uccsd_fermion=uccsd_singlet_generator(initial_amplitudes,qubit,elektron)
#b=uccsd_generator(initial_amplitudes,qubit,elektron)
print(uccsd_fermion)

-0.05677 [0^ 2 1^ 3] +
-0.05677 [1^ 3 0^ 2] +
0.05677 [2^ 0 3^ 1] +
0.05677 [3^ 1 2^ 0]


In [5]:
uccsd_bk=bravyi_kitaev(uccsd_fermion)
print(uccsd_bk)

-0.0141925j [X0 Z1 Y2] +
-0.0141925j [X0 Z1 Y2 Z3] +
-0.0141925j [X0 Y2] +
-0.0141925j [X0 Y2 Z3] +
0.0141925j [Y0 Z1 X2] +
0.0141925j [Y0 Z1 X2 Z3] +
0.0141925j [Y0 X2] +
0.0141925j [Y0 X2 Z3]


In [6]:
import openfermion as of
import cirq
import openfermioncirq as ofc
import numpy as np
class MyAnsatz(ofc.VariationalAnsatz):
    def params(self):
        '''Parameter of the ansatz'''
        return [cirq.Symbol('theta_0'),cirq.Symbol('theta_1')]
    def operations(self, qubits):
        '''Ansatz Prosedur'''
        q0, q1, q2, q3 = qubits
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q2)
        yield cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
         
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.CNOT(q0,q2)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q2)
        yield cirq.CNOT(q0,q2)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q2)
        yield cirq.CNOT(q0,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q0,q2)
        yield cirq.H(q0)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q2)
        
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q2)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q2)
        yield cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.H(q2)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q2)
        yield cirq.CNOT(q0,q1),cirq.CNOT(q1,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q1,q2),cirq.CNOT(q0,q1)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.H(q2)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q2)
        yield cirq.CNOT(q0,q2)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q2)
        yield cirq.CNOT(q0,q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.H(q2)
        
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q2)
        yield cirq.CNOT(q0,q2),cirq.CNOT(q2,q3)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_1')).on(q3)
        yield cirq.CNOT(q2,q3),cirq.CNOT(q0,q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.H(q2)
        
        
    def _generate_qubits(set):
        return cirq.LineQubit.range(4)
        

In [7]:
ansatz=MyAnsatz()
print(ansatz.circuit)

0: ───H──────────@───────────────────────@─────────H──────────H───@───────────────────────────────@─────────H──────────H───@───────────────@───H─────────H──────────@───────────────────────@───H─────────X^(-3/7)───@───────────────────────@───X^(3/7)───X^(-3/7)───@───────────────────────────────@───X^(3/7)───X^(-3/7)───@───────────────@───X^(3/7)───X^(-3/7)───@───────────────────────@───X^(3/7)───
                 │                       │                        │                               │                        │               │                        │                       │                        │                       │                        │                               │                        │               │                        │                       │
1: ──────────────X───@───────────────@───X────────────────────────X───@───────────────────────@───X────────────────────────┼───────────────┼────────────────────────┼───────────────────────┼────────────────────────X─

In [8]:
import openfermionpyscf as ofpyscf
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
geometry = [('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.7414))]
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Perform electronic structure calculations and
# obtain Hamiltonian as an InteractionOperator
hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge)
hamiltonian_bk=bravyi_kitaev(get_fermion_operator(hamiltonian))

In [9]:
objective = ofc.HamiltonianObjective(hamiltonian_bk)


q0, q1, _, _ = ansatz.qubits
reference_circuit = cirq.Circuit.from_ops(cirq.X(q0), cirq.X(q1))


study = ofc.VariationalStudy(
    name='my_hydrogen_study',
    ansatz=ansatz,
    objective=objective,
    preparation_circuit=reference_circuit)

print(study.num_params)
print(ansatz.default_initial_params())

2
[0. 0.]


In [10]:
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'nelder-mead'},
    options={'maxiter': 100,'disp':True})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=ansatz.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Optimization terminated successfully.
         Current function value: -0.351190
         Iterations: 5
         Function evaluations: 13
Nilai optimum: -0.3511901986746766
parameter optimasinya:
[ 0.00025  -0.000125]


In [12]:
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'SLSQP'},
    options={'maxiter': 100,'disp':True})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=ansatz.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.35119019867467627
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1
Nilai optimum: -0.35119019867467627
parameter optimasinya:
[0. 0.]


In [13]:
algorithm = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'powell'},
    options={'maxiter': 100,'disp':True})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algorithm,
    initial_guess=ansatz.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Optimization terminated successfully.
         Current function value: -0.351190
         Iterations: 1
         Function evaluations: 26
Nilai optimum: -0.35119019867467677
parameter optimasinya:
[3.02903489 0.47213597]
